<a href="https://colab.research.google.com/github/Deng-Xian-Sheng/Real-technology/blob/main/%E2%80%9CDeepSeek_R1_Distill_Qwen_1_5B_Conversational_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

무료 Tesla T4 Google Colab 인스턴스에서 이 프로그램을 실행하려면 '런타임' > '모두 실행'을 클릭하세요.

   도움이 필요하시면 Discord에 참여하시고 Github에서 ⭐로 표시해주세요.
자신의 컴퓨터에 Unsloth를 설치하려면 [GitHub 페이지](redacted link)의 설치 지침을 따르세요.

이 노트북에서는 다음을 배울 수 있습니다: [redacted link], [redacted link], [redacted link], [redacted link].

문서에서 모든 [redacted link]와 [redacted link]를 확인하세요.

### 최신 소식

[신규] Phi-4 모델의 여러 버그를 수정하여 정확도가 크게 향상되었습니다. [redacted link] 참조.

[신규] GGUF 포함 모든 Phi-4 모델 업로드 확인: [redacted link]

[신규] 2024년 11월 기준 Unsloth는 [redacted link] 지원!

추가 질문이 있으면 알려주세요.

### 설치

In [ ]:
%%capture
# 일반적으로 pip install unsloth로 충분합니다

# 2025년 1월 31일 기준 Colab의 Pytorch 관련 일시적 문제
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

### Unsloth 설정

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # RoPE 스케일링 자동 지원
dtype = None  # 자동 감지 (T4/V100: float16, Ampere+: bfloat16)
load_in_4bit = True  # 4비트 양자화 사용

# 4비트 사전 양자화 모델 목록
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Mistral-Small-Instruct-2409",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-27b-bnb-4bit",
    "unsloth/Llama-3.2-1B-bnb-4bit",
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
]

qwen_models = [
    "unsloth/Qwen2.5-Coder-32B-Instruct",
    "unsloth/Qwen2.5-Coder-7B",
    "unsloth/Qwen2.5-14B-Instruct",
    "unsloth/Qwen2.5-7B",
    "unsloth/Qwen2.5-72B-Instruct",
    "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit"
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

LoRA 어댑터 추가 (전체 파라미터의 1-10%만 업데이트)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",  # 긴 컨텍스트 시 사용
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

## 데이터 준비
Qwen-2.5 형식의 대화 스타일 파인튜닝을 위해 mlabonne/FineTome-100k 데이터셋 사용

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

In [ ]:
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# JSON 데이터셋 로드 예제
from datasets import load_dataset
dataset = load_dataset("json",data_files='./drive/MyDrive/Colab Notebooks/xiaoshuo_dataset.json', split = "train")

ShareGPT 형식을 HuggingFace 표준 형식으로 변환

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = dataset.map(formatting_prompts_func, batched = True,)

데이터 구조 확인

In [ ]:
dataset[5]["conversations"]

In [ ]:
dataset[5]["text"]

# 모델 학습
Huggingface TRL의 SFTTrainer 사용

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=4,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=30,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="paged_adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. 최대 메모리 = {max_memory} GB.")
print(f"예약된 메모리: {start_gpu_memory} GB.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title 최종 메모리 및 시간 통계 표시
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} 초가 학습에 사용되었습니다.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} 분이 학습에 사용되었습니다."
)
print(f"최대 예약 메모리 = {used_memory} GB.")
print(f"학습을 위한 최대 예약 메모리 = {used_memory_for_lora} GB.")
print(f"최대 메모리의 최대 예약 메모리 % = {used_percentage} %.")
print(f"학습을 위한 최대 예약 메모리 % = {lora_percentage} %.")

## 추론
모델을 실행해 봅시다! 지침과 입력을 변경할 수 있습니다. 출력을 비워 두세요!

[신규 기능] 무료 Colab에서 Llama-3.1 8b Instruct를 사용하여 2배 빠른 추론을 시도해 보세요 [여기에](삭제된 링크)

min_p = 0.1 및 temperature = 1.5를 사용합니다. 자세한 내용은 이 [트윗](삭제된 링크)을 참조하세요.

In [ ]:
from unsloth.chat_templates import get_chat_template  # unsloth 라이브러리에서 get_chat_template 함수 가져오기

# Qwen-2.5의 채팅 템플릿을 가져와 토크나이저에 적용
tokenizer = get_chat_template(
    tokenizer,
    chat_template="qwen-2.5",
)

FastLanguageModel.for_inference(model)  # 기본 2배 빠른 추론 활성화

# 사용자 역할과 내용을 포함하는 메시지 목록 정의
messages = [
    {"role": "user", "content": "피보나치 수열을 계속하세요: 1, 1, 2, 3, 5, 8,"},  # 사용자 요청: 피보나치 수열 계속
]

# 채팅 템플릿을 사용하여 메시지를 처리하고, 토큰화하고, 생성 프롬프트를 추가하고, PyTorch 텐서로 변환
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,  # 생성을 위해 반드시 추가해야 함
    return_tensors="pt",
).to("cuda")  # 텐서를 CUDA 장치(GPU)로 이동

# 모델을 사용하여 출력 생성
outputs = model.generate(
    input_ids=inputs,  # 입력 ID
    max_new_tokens=64,  # 최대 생성할 새 토큰 수
    use_cache=True,  # 캐시를 사용하여 생성 속도 향상
    temperature=1.5,  # 생성 결과의 무작위성 제어
    min_p=0.1,  # 생성 결과의 다양성 제어
)

# 모델 출력을 텍스트로 디코딩
tokenizer.batch_decode(outputs)

TextStreamer를 사용하여 연속 추론을 수행할 수도 있습니다. 이렇게 하면 전체 생성 결과를 기다리지 않고 토큰별로 결과를 볼 수 있습니다!

In [ ]:
FastLanguageModel.for_inference(model) # 기본 2배 빠른 추론 활성화

messages = [
    {"role": "user", "content": "피보나치 수열을 계속하세요: 1, 1, 2, 3, 5, 8,"},  # 사용자 메시지: 피보나치 수열 계속
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,  # 메시지 토큰화
    add_generation_prompt = True,  # 생성을 위해 반드시 추가해야 함
    return_tensors = "pt",  # PyTorch 텐서 반환
).to("cuda")  # 텐서를 CUDA 장치(GPU)로 이동

from transformers import TextStreamer  # 연속 추론을 위한 TextStreamer 가져오기
text_streamer = TextStreamer(tokenizer, skip_prompt = True)  # TextStreamer 인스턴스 생성
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,  # 모델을 사용하여 출력 생성
                   use_cache = True, temperature = 1.5, min_p = 0.1)  # 생성 매개변수 설정

### 미세 조정된 모델 저장 및 로드
최종 모델을 LoRA 어댑터로 저장하려면 Huggingface의 push_to_hub를 사용하여 온라인으로 저장하거나 save_pretrained를 사용하여 로컬로 저장할 수 있습니다.

[참고] 이는 전체 모델이 아닌 LoRA 어댑터만 저장합니다. 16비트 또는 GGUF 형식으로 저장하려면 아래로 스크롤하세요!

In [ ]:
model.save_pretrained("lora_model")  # 로컬 저장
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # 온라인 저장
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # 온라인 저장

이제, 방금 저장한 LoRA 어댑터를 추론에 사용하려면 False를 True로 설정하세요:

이제, 방금 저장한 LoRA 어댑터를 추론에 사용하려면 False를 True로 설정하세요:

In [ ]:
if False:  # 이 코드는 조건이 False이므로 현재 실행되지 않습니다.
    from unsloth import FastLanguageModel  # unsloth 라이브러리에서 FastLanguageModel 클래스 가져오기
    # 사전 훈련된 모델에서 모델과 토크나이저 로드
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="lora_model",  # 훈련에 사용한 모델 이름, 실제 모델 이름으로 바꿔야 합니다.
        max_seq_length=max_seq_length,  # 최대 시퀀스 길이, 이전에 정의됨
        dtype=dtype,  # 데이터 타입, 이전에 정의됨
        load_in_4bit=load_in_4bit,  # 4비트 양자화로 로드할지 여부, 이전에 정의됨
    )
    FastLanguageModel.for_inference(model)  # 네이티브 2배속 추론 활성화

# 메시지 목록 정의, 사용자 역할 및 콘텐츠 포함
messages = [
    {"role": "user", "content": "프랑스 수도의 높은 탑을 묘사하세요."},  # 사용자 메시지: 프랑스 수도의 높은 탑 묘사
]

# 채팅 템플릿을 사용하여 메시지 처리, 토큰화, 생성 프롬프트 추가, PyTorch 텐서로 변환
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,  # 메시지 토큰화
    add_generation_prompt=True,  # 생성을 위해 반드시 추가해야 함
    return_tensors="pt",  # PyTorch 텐서 반환
).to("cuda")  # 텐서를 CUDA 장치(GPU)로 이동

from transformers import TextStreamer  # TextStreamer를 임포트하여 연속 추론에 사용
text_streamer = TextStreamer(tokenizer, skip_prompt=True)  # TextStreamer 인스턴스 생성
# 모델을 사용하여 출력 생성
_ = model.generate(
    input_ids=inputs,  # 입력 ID
    streamer=text_streamer,  # TextStreamer를 사용하여 연속 추론
    max_new_tokens=128,  # 최대 생성할 새 토큰 수
    use_cache=True,  # 캐시를 사용하여 생성 속도 향상
    temperature=1.5,  # 생성 결과의 무작위성 제어
    min_p=0.1,  # 생성 결과의 다양성 제어
)

Hugging Face의 AutoModelForPeftCausalLM을 사용할 수도 있습니다. Unsloth를 설치하지 않은 경우에만 이 방법을 사용하십시오. 4비트 모델 다운로드를 지원하지 않으므로 매우 느릴 수 있으며 Unsloth의 추론 속도는 2배 빠릅니다.

In [ ]:
if False:  # 이 코드는 조건이 False이므로 현재 실행되지 않습니다.
    # 저는 이것을 강력히 권장하지 않습니다. 가능하다면 Unsloth를 사용하십시오.
    from peft import AutoPeftModelForCausalLM  # peft 라이브러리에서 AutoPeftModelForCausalLM 클래스 가져오기
    from transformers import AutoTokenizer  # transformers 라이브러리에서 AutoTokenizer 클래스 가져오기

    # 사전 훈련된 모델에서 모델 로드, LoRA 어댑터 사용
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model",  # 훈련에 사용한 모델 이름, 실제 모델 이름으로 바꿔야 합니다.
        load_in_4bit=load_in_4bit,  # 4비트 양자화로 로드할지 여부, 이전에 정의됨
    )
    # 사전 훈련된 모델에서 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained("lora_model")  # 훈련에 사용한 모델 이름, 실제 모델 이름으로 바꿔야 합니다.

## VLLM을 위해 float16 형식으로 저장
float16 형식으로 직접 저장하는 것도 지원합니다. float16의 경우 merged_16bit를 선택하고 int4의 경우 merged_4bit를 선택하십시오. 또한 LoRA 어댑터를 대체 옵션으로 사용할 수도 있습니다. push_to_hub_merged를 사용하여 Hugging Face 계정에 업로드하십시오! 개인 토큰은 [redacted link]에서 얻을 수 있습니다.

In [ ]:
# 16비트로 병합
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# 4비트로 병합
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# LoRA 어댑터만
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

## GGUF / llama.cpp 변환
GGUF / llama.cpp에 저장하기 위해 이제 기본적으로 지원합니다! llama.cpp를 복제했으며 기본적으로 q8_0으로 저장합니다. q4_k_m과 같은 모든 방법을 허용합니다. save_pretrained_gguf를 사용하여 로컬에 저장하고 push_to_hub_gguf를 사용하여 HF에 업로드합니다.

지원되는 양자화 방법 몇 가지 (전체 목록은 [redacted link] 참조):

 - q8_0 - 빠른 변환. 리소스 점유율이 높지만 일반적으로 허용됩니다.
 - q4_k_m - 권장. attention.wv 및 feed_forward.w2 텐서의 절반에는 Q6_K를 사용하고 그렇지 않으면 Q4_K를 사용합니다.
 - q5_k_m - 권장. attention.wv 및 feed_forward.w2 텐서의 절반에는 Q6_K를 사용하고 그렇지 않으면 Q5_K를 사용합니다.

[신규] Ollama에 미세 조정하고 자동 내보내려면 [redacted link]를 사용해 보십시오.

In [ ]:
# 8비트 Q8_0 형식으로 저장
if False: model.save_pretrained_gguf("model", tokenizer,)
# 토큰을 얻으려면 https://huggingface.co/settings/tokens를 방문하는 것을 잊지 마십시오!
# 그리고 hf를 사용자 이름으로 바꾸십시오!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# 16비트 GGUF 형식으로 저장
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# q4_k_m GGUF 형식으로 저장
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# 여러 GGUF 형식으로 저장 - 여러 형식이 필요한 경우 더 빠릅니다!
if False:
    model.push_to_hub_gguf(
        "hf/model", # hf를 사용자 이름으로 변경하십시오!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # https://huggingface.co/settings/tokens에서 토큰을 얻습니다.
    )

이제 llama.cpp 또는 Jan 또는 Open WebUI와 같은 UI 기반 시스템에서 model-unsloth.gguf 파일 또는 model-unsloth-Q4_K_M.gguf 파일을 사용하십시오. [redacted link]에서 Jan을 설치하고, [redacted link]에서 Open WebUI를 설치할 수 있습니다.
끝났습니다! Unsloth에 대한 질문이 있으면 [redacted link] 채널이 있습니다! 오류가 발생하거나 최신 LLM 정보를 알고 싶거나 도움이 필요하거나 프로젝트에 참여하는 등 언제든지 Discord에 참여하십시오!

기타 링크:

Llama 3.2 대화 노트북. [redacted link]
미세 조정 결과를 Ollama에 저장. [redacted link]
Llama 3.2 시각적 미세 조정 - 방사선학 사용 사례. [redacted link]
DPO, ORPO, 지속적인 사전 훈련, 대화형 미세 조정 등에 대한 노트북은 [redacted link]에서 확인하십시오!
  
도움이 필요하면 Discord에 참여하고 Github에서 ⭐️로 응원해 주세요. ⭐️